<a href="https://colab.research.google.com/github/ngngocsonan2610/note/blob/master/ML_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Preprocessing](#scrollTo=-2COeSc5ml1_)

>>[Transformation Scaling/Standardize](#scrollTo=PcX1fpPrrg-_)

>>[Encoding](#scrollTo=qeLaVzFW0Se6)

>>>[Onehot Encode](#scrollTo=HULldo6Zm6l1)

>>>[Label Encode](#scrollTo=Hz-vTHED0Vbc)

>>>[Reduce memory](#scrollTo=WzFvPys_F9KB)

>>>[Reduce dimensional checking](#scrollTo=a_a8z7m7uec_)

>>[Feature Generation](#scrollTo=ffkEEAsbC4EW)

>>[Feature Selection](#scrollTo=LfdUhklbDAQa)

>[MATH REVIEW](#scrollTo=OxKUSl9g2BWM)

>[Regression Algorithms](#scrollTo=8FIjZqKbt8P8)

>>[Linear Regression](#scrollTo=dpc1c9ZsP2i6)

>>[Perceptron Learning Algorithm (PLA)](#scrollTo=eX6sM5ldJu0-)

>>[Logistic Regression](#scrollTo=bwn5AC4UPgJE)

>>[Softmax Regression](#scrollTo=IDHLmr9DPmak)

>>[SVM](#scrollTo=4Kz3zCJNjApN)

>[2.Instance-based (memory-based)](#scrollTo=_j1ygYkdutFv)

>>[K-nearest neighbors (KNN)](#scrollTo=m1MEZV-Js4Yq)

>[3.Clustering Algorithms](#scrollTo=np0HDTN556XV)

>>[K-means clustering](#scrollTo=8OJRGMNB5xXU)

>[4.Reduce Dimension](#scrollTo=W8gx540rhd0Q)

>>>[Singular Value Decomposition (SVD)](#scrollTo=ZF025S_Lgd07)

>>[Principal Component Analysis](#scrollTo=Xc9w3obZjBTW)



# Preprocessing

## Transformation Scaling/Standardize
News:
- https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing


Căn bản:
- [Scale, Standardize, or Normalize with Scikit-Learn](https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02)
![alt text](https://miro.medium.com/max/4800/1*z-C9ANBC4rjsk-ZK4wzijg.png)
- [Why, How and When to Scale your Features](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e)


- Đối với từng model nên xét thêm





## Encoding


### Onehot Encode

### Label Encode

### Reduce memory

In [0]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

### Reduce dimensional checking


In [0]:
#scaling only on tranform data
df_V = df_V[c_V_trans] 
df_V_test = df_V_test[c_V_trans]

#standardize scaled
scaler = StandardScaler().fit(df_V)
df_V_scaled = scaler.transform(df_V)
print('Scaled mean df_V',df_V_scaled[:,0].mean())  # zero (or very close)
print('Scaled std df_V',df_V_scaled[:,0].std()) 
scaler = StandardScaler().fit(df_V_test)
df_V_test_scaled = scaler.transform(df_V_test)
print('Scaled mean df_V',df_V_test_scaled[:,0].mean())  # zero (or very close)
print('Scaled std df_V',df_V_test_scaled[:,0].std()) 

# plot cumulative explained variance
# pca = PCA().fit(df_V_scaled)
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# #plt.xlim(0,7,1)
# plt.xlabel('Number of components')
# plt.ylabel('Cumulative explained variance')


# Setup Principal component analysis
pca = PCA(n_components=125) 
#pca = PCA(n_components=0.96)  #v2: n_components=0.95
df_V_pca = pca.fit_transform(df_V_scaled)
np.save('df_V_pca_v3.npy',df_V_pca)
#df_V_pca.to_csv('df_V_pca', sep='\t')
df_V_test_pca = pca.fit_transform(df_V_test_scaled)
np.save('df_V_test_pca_v3.npy',df_V_test_pca)


In [0]:
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

'''
Scatter plot giữa 3 cột bất kỳ trong dữ liệu, được hue = TARGET
cho cái nhìn về dữ liệu, xem liệu nó có khả năng phân tách không
'''

trace1 = go.Scatter3d(x = churn["MEMBER_ANNUAL_INCOME"],
                      y = churn["ANNUAL_FEES"],
                      z = churn["MEMBERSHIP_TERM_YEARS"],
                      mode = "markers",
                      name = "Churn customers",
                      text = "Id : " + churn["MEMBERSHIP_NUMBER"],
                      marker = dict(size = 1,color = "red")
                     )
trace2 = go.Scatter3d(x = not_churn["MEMBER_ANNUAL_INCOME"],
                      y = not_churn["ANNUAL_FEES"],
                      z = not_churn["MEMBERSHIP_TERM_YEARS"],
                      name = "Non churn customers",
                      text = "Id : " + not_churn["MEMBERSHIP_NUMBER"],
                      mode = "markers",
                      marker = dict(size = 1,color= "green")
                     )

layout = go.Layout(dict(title = "Monthly charges,total charges & tenure in customer attrition",
                        scene = dict(camera = dict(up=dict(x= 0 , y=0, z=0),
                                                   center=dict(x=0, y=0, z=0),
                                                   eye=dict(x=1.25, y=1.25, z=1.25)),
                                     xaxis  = dict(title = "annual incomes",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'),
                                     yaxis  = dict(title = "annual fees",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'
                                                  ),
                                     zaxis  = dict(title = "term years",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'
                                                  )
                                    ),
                        height = 700,
                       )
                  )
                  

data = [trace1,trace2]
fig  = go.Figure(data = data,layout = layout)
py.iplot(fig)

In [0]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)

X = tmp[[i for i in tmp.columns if i not in Id_col + target_col]]
Y = tmp[target_col + Id_col]

principal_components = pca.fit_transform(X)
pca_data = pd.DataFrame(principal_components,columns = ["PC1","PC2"])
pca_data = pca_data.merge(Y,left_index=True,right_index=True,how="left")
pca_data["CHURN"] = pca_data["CHURN"].replace({1:"CANCELLED",0:"INFORCE"})

def pca_scatter(target,color) :
    tracer = go.Scatter(x = pca_data[pca_data["CHURN"] == target]["PC1"] ,
                        y = pca_data[pca_data["CHURN"] == target]["PC2"],
                        name = target,mode = "markers",
                        marker = dict(color = color,
                                      line = dict(width = .5),
                                      symbol =  "diamond-open"),
                        text = ("Customer Id : " + 
                                pca_data[pca_data["CHURN"] == target]['MEMBERSHIP_NUMBER'])
                       )
    return tracer

layout = go.Layout(dict(title = "Visualising data with principal components",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "principal component 1",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "principal component 2",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        height = 600
                       )
                  )
trace1 = pca_scatter("CANCELLED",'red')
trace2 = pca_scatter("INFORCE",'royalblue')
data = [trace2,trace1]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

## Feature Generation

[Simple FE](https://machinelearningcoban.com/general/2017/02/06/featureengineering/)
-    Trực tiếp lấy raw data
-    Bag-of-words
  -    Bag-of-Words trong Computer Vision
-    Feature Scaling and Normalization
  -        Rescaling
  -        Standardization
  -        Scaling to unit length

  - Feature Generation: ở bước này tập trung các kĩ thuật để tạo ra feature: xử lí nlp, image, binning, scaling, grouping, aggregate.
  - Kết hợp với Modelling để tìm ra nhóm feature tốt và tập trung mạnh vào đó


## Feature Selection
- Feature Selection: ở bước này tập trung các kĩ thuật chọn feature để tốt ưu mô hình
  - Chọn được feature tốt và tối ưu được bộ nhớ
  - Chọn được model parameters tốt nhất phù hợp với bộ features tốt nhất
  - Kết hợp với các kĩ thuật chia dataset (CV, leave-one)



- Tiêu chí đánh giá **metric of feature**
  - AUC: thể hiện khả năng dự đoán
  - Correlation: kiểm tra độ tương quan giữa feature với target hoặc với các feature quan trọng khác
  - Converage: kiểm tra null, null nhiều thì ít thông tin
  - Weighted of evidence, and information value:
    - $\ln \left(\frac{P(\text {Good})}{P(\text {Bad})}\right)$
    - \begin{array}{l}{\text { Information value }} \\ {\qquad \sum(P(G o o d)-P(B a d)) * \ln \left(\frac{P(G o d)}{P(B a d)}\right)}\end{array}



[Brute Force Approach](https://www.kdnuggets.com/2017/11/rapidminer-basic-concepts-feature-selection.html)
Cách tiếp cận bằng các chạy thử thất cả các feature combination và so sánh trên các metrics of feature

[Feature Selection](https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/) 
- Forwarding: ta bắt đầu với tập feature rỗng, sau đó lần lượt add thêm feature vào tập này. Nếu thấy performance của model tăng ta sẽ tiếp tục quá trình này, ngược lại sẽ dừng lại.
- Backwarding: ta bắt đầu với toàn bộ tập feature, sau đó lần lượt remove từng feature khỏi tập này. Nếu thấy performance của model tăng hoặc giảm không quá nhiều, ta sẽ tiếp tục quá trình này, ngược lại nếu performance bị drop quá mạnh sẽ dừng lại.
- Hybridge: kết hợp cả 2 hướng trên


Research:
- [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection)
- [2](https://www.kaggle.com/sz8416/6-ways-for-feature-selection) example selectkbes-RFE in kaggle
- [3](https://www.kaggle.com/dkim1992/feature-selection-ranking) chua doc
- [4](https://machinelearningmastery.com/feature-selection-machine-learning-python/) tong quat
- [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html)
- [5](https://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/) chua doc

**RFE**

[theory](https://www.kaggle.com/nroman/recursive-feature-elimination)

Trong mini course này, tôi sẽ áp dụng hướng “backwarding”. Các bước thực hiện như sau:

- Đặt: n là số lần lặp feature selection, k là số feature sẽ drop ở mỗi lần lặp, p là AUC sau mỗi lần train
Train model với XGboost
- Lấy kết quả feature important sắp xếp giảm dần và loại ra k feature có giá trị thấp nhất
- Lưu lại performance hiện tại để so sánh với performance tiếp theo.
- Nếu thấp hơn ngưỡng p sẽ dừng
- Tiếp tục quá trình selection

Tuỳ theo số lượng feature và cài đặt hyper-parameter của model thì thời gian sẽ nhanh chậm khác nhau.

Additional:
- Genetic algorithm for feature selection

Source:
- [1](https://ongxuanhong.wordpress.com/2019/04/17/data-science-mini-course/#more-15645)
- [2](https://towardsdatascience.com/feature-selection-in-python-recursive-feature-elimination-19f1c39b8d15) doc them

---
**Boruta**
[1](http://danielhomola.com/2015/05/08/borutapy-an-all-relevant-feature-selection-method/)

# [MATH REVIEW](https://colab.research.google.com/github/ngngocsonan2610/note/blob/master/4.Math.ipynb)

# 1.Regression Algorithms

## Linear Regression


Lost function: MSE
Giải đạo hàm: 
![alt text](https://cdn.mathpix.com/snip/images/FK2Cy8U9_QURngdhSj25RQTfGExgynPNUPK0pWZTz08.original.fullsize.png)
![alt text](https://cdn.mathpix.com/snip/images/C9R4s0LujO1M6IRNd6SfTkvsrkVTPzcy7J2-PUeuE1s.original.fullsize.png)

**Discuss & Code**
**Hạn chế:**
- Linear Regression là nó rất nhạy cảm với nhiễu (sensitive to noise)
- Hạn chế thứ 2 là nó không biểu diễn được các mô hình phcứ tạp. Chúng ta biết rằng LR vẫn có thể áp dụng khi relationship giữa input và outcome là không tuyến tính, nhưng rất khó để tìm được các đặc trựng (các distribution) của từng điểm $x$
- Trong trường hợp ma trận $A=X X^{T}$ không khả nghich:  Ta biến đổi $A=X X^{T}+\lambda I$ với $\lambda$ là 1 số dương rất nhỏ và ma trận đơn vị $I$. Giải nghiệm ta được,  $\mathbf{w}=\left(\mathbf{X X}^{T}+\lambda \mathbf{I}\right)^{-1} \mathbf{X} \mathbf{y}$  - còn gọi là **Ridge regression**: giúp phương trình đạo hàm có nghiệm duy nhất và tránh overfittting.
- Không thể giải phương trình đạo hàm: Gradient descent








In [0]:
    
from IPython.display import IFrame

IFrame(src='https://nbviewer.jupyter.org/github/tiepvupsu/tiepvupsu.github.io/blob/master/assets/LR/LR.ipynb?flush_cache=true',
       width=800, height=600)

## Perceptron Learning Algorithm (PLA)
Bài toán Perceptron được phát biểu như sau: Cho hai class được gán nhãn, hãy tìm một đường phẳng sao cho toàn bộ các điểm thuộc class 1 nằm về 1 phía, toàn bộ các điểm thuộc class 2 nằm về phía còn lại của đường phẳng đó. Với giả định rằng tồn tại một đường phẳng như thế.

Nếu tồn tại một đường phẳng phân chia hai class thì ta gọi hai class đó là linearly separable. Các thuật toán classification tạo ra các boundary là các đường phẳng được gọi chung là Linear Classifier.

Source:
[1](https://machinelearningcoban.com/2017/01/21/perceptron/),

>**Thuật toán PLA**
1. Chọn ngẫu nhiên một vector hệ số $w$ với các phần tử gần 0.
2. Duyệt ngẫu nhiên qua từng điểm dữ liệu $x_i$:
Nếu xi
  *   Nễu $x_i$ được phân lớp đúng, tức $sgn(w^Tx_i)=y_i$, chúng ta không làm gì
  *   Nếu $x_i$ bị misclassifed, cập nhật $w = w + y_i x_i$
3. Kiểm tra xem có bao nhiêu điểm bị misclassifed. Nếu không còn điểm nào, dừng thuật toán. Nếu còn, quay lại bước 2.

**Discuss & code**
**Hạn chế**:
* PLA có thể cho vô số nghiệm khác nhau: rõ ràng rằng, nếu hai class là linearly separable thì có vô số đường thằng phân cách 2 class đó.
* PLA đòi hỏi dữ liệu linearly separable: Hai class trong ví dụ dưới đây tương đối linearly separable. Mỗi class có 1 điểm coi như nhiễu nằm trong khu vực các điểm của class kia. PLA sẽ không làm việc trong trường hợp này vì luôn luôn có ít nhất 2 điểm bị misclassified.

**Cải thiện**
* Pocket Algorithm
Một cách tự nhiên, nếu có một vài nhiễu, ta sẽ đi tìm một đường thẳng phân chia hai class sao cho có ít điểm bị misclassified nhất. Việc này có thể được thực hiện thông qua PLA với một chút thay đổi nhỏ như sau:
  1. Giới hạn số lượng vòng lặp của PLA.
  2. Mỗi lần cập nhật nghiệm $w$ mới, ta đếm xem có bao nhiêu điểm bị misclassified. Nếu là lần đầu tiên, giữ lại nghiệm này trong pocket (túi quần). Nếu không, so sánh số điểm misclassified này với số điểm misclassified của nghiệm trong pocket, nếu nhỏ hơn thì lôi nghiệm cũ ra, đặt nghiệm mới này vào.

Thuật toán này giống với thuật toán tìm phần tử nhỏ nhất trong 1 mảng.

**Code**
```python
def h(w, x):    
    return np.sign(np.dot(w.T, x))
```

[1](https://nbviewer.jupyter.org/github/tiepvupsu/tiepvupsu.github.io/blob/master/assets/pla/perceptron.py)

## Logistic Regression
Loss function: maximum likelihood

Lấy logarit tự nhiên (cơ số e) của likelihood function biến phép nhân thành phép cộng và để tránh việc số quá nhỏ. Sau đó lấy ngược dấu để được một hàm và coi nó là hàm mất mát. Lúc này bài toán tìm giá trị lớn nhất (maximum likelihood) trở thành bài toán tìm giá trị nhỏ nhất của hàm mất mát (hàm này còn được gọi là negative log likelihood):


$$
J\left(\mathbf{w} ; \mathbf{x}_{i}, y_{i}\right)=-\left(y_{i} \log z_{i}+\left(1-y_{i}\right) \log \left(1-z_{i}\right)\right)\\
\frac{\partial J\left(\mathbf{w} ; \mathbf{x}_{i}, y_{i}\right)}{\partial \mathbf{w}}=\left(z_{i}-y_{i}\right) \mathbf{x}_{i}
$$


Source: [1](https://machinelearningcoban.com/2017/01/27/logisticregression/)

**Discuss & code**

- Một điểm cộng cho Logistic Regression so với PLA là nó không cần có giả thiết dữ liệu hai class là linearly separable. Tuy nhiên, boundary tìm được vẫn có dạng tuyến tính. Vậy nên mô hình này chỉ phù hợp với loại dữ liệu mà hai class là gần với linearly separable. 
- Một kiểu dữ liệu mà Logistic Regression không làm việc được là dữ liệu mà một class chứa các điểm nằm trong 1 vòng tròn, class kia chứa các điểm bên ngoài đường tròn đó. Kiểu dữ liệu này được gọi là phi tuyến (non-linear). Sau một vài bài nữa, tôi sẽ giới thiệu với các bạn các mô hình khác phù hợp hơn với loại dữ liệu này hơn.
- Một hạn chế nữa của Logistic Regression là nó yêu cầu các điểm dữ liệu được tạo ra một cách độc lập với nhau. Trên thực tế, các điểm dữ liệu có thể bị ảnh hưởng bởi nhau. Ví dụ: có một nhóm ôn tập với nhau trong 4 giờ, cả nhóm đều thi đỗ (giả sử các bạn này học rất tập trung), nhưng có một sinh viên học một mình cũng trong 4 giờ thì xác suất thi đỗ thấp hơn. Mặc dù vậy, để cho đơn giản, khi xây dựng mô hình, người ta vẫn thường giả sử các điểm dữ liệu là độc lập với nhau.


**Code**
```python
def sigmoid(s):
    return 1/(1 + np.exp(-s))
```

## Softmax Regression
Loss function: xây dựng bởi entropy
$$J(\mathbf{W} ; \mathbf{X}, \mathbf{Y})=-\sum_{i=1}^{N} \sum_{j=1}^{C} y_{j i} \log \left(a_{j i}\right)=-\sum_{i=1}^{N} \sum_{j=1}^{C} y_{j i} \log \left(\frac{\exp \left(\mathbf{w}_{j}^{T} \mathbf{x}_{i}\right)}{\sum_{k=1}^{C} \exp \left(\mathbf{w}_{k}^{T} \mathbf{x}_{i}\right)}\right)\\
\frac{\partial J(\mathbf{W})}{\partial \mathbf{W}}=\sum_{i=1}^{N} \mathbf{x}_{i} \mathbf{e}_{i}^{T}=\mathbf{X} \mathbf{E}^{T}\\
where,\\
\mathbf{E}=\mathbf{A}-\mathbf{Y}\\
a_{i}=\frac{\exp \left(z_{i}\right)}{\sum_{j=1}^{C} \exp \left(z_{j}\right)}, \quad \forall i=1,2, \ldots, C
$$

Khi $C=2$, Softmax Regression và Logistic Regression là giống nhau

Source: 
[1](https://machinelearningcoban.com/2017/02/17/softmax/),

**Code**
```python
import numpy as np 
def softmax(Z):
    """
    Compute softmax values for each sets of scores in V,   each column of V is a set of score.    
    """
    e_Z = np.exp(Z)
    A = e_Z / e_Z.sum(axis = 0)
    return A

def softmax_stable(Z):
    """
    Compute softmax values for each sets of scores in Z.
    each column of Z is a set of score.    
    """
    e_Z = np.exp(Z - np.max(Z, axis = 0, keepdims = True))
    A = e_Z / e_Z.sum(axis = 0)
    return A
 
  
```

## SVM
Support vector machine
* Với bài toán binary classification mà 2 classes là linearly separable, có vô số các siêu mặt phẳng giúp phân biệt hai classes, tức mặt phân cách. Với mỗi mặt phân cách, ta có một classifier. Khoảng cách gần nhất từ 1 điểm dữ liệu tới mặt phân cách ấy được gọi là margin của classifier đó.
* Việc này có thể được tổng quát lên không gian nhiều chiều: Khoảng cách từ một điểm (vector) có toạ độ $\mathbf{x_0}$ tới siêu mặt phẳng (hyperplane) có phương trình $\mathbf{w}^{T} \mathbf{x}+b=0$ được xác định bởi: 
$$\frac{\left|\mathbf{w}^{T} \mathbf{x}_{0}+b\right|}{\|\mathbf{w}\|_{2}}$$
    với $\|\mathbf{w}\|_{2}=\sqrt{\sum_{i=1}^{d} w_{i}^{2}}$ , d là số chiều không gian
* Support Vector Machine là bài toán đi tìm mặt phân cách sao cho margin tìm được là lớn nhất, đồng nghĩa với việc các điểm dữ liệu an toàn nhất so với mặt phân cách.
* Bài toán tối ưu trong SVM là một bài toán lồi với hàm mục tiêu là stricly convex, nghiệm của bài toán này là duy nhất. Hơn nữa, bài toán tối ưu đó là một Quadratic Programming (QP).
* Mặc dù có thể trực tiếp giải SVM qua bài toán tối ưu gốc này, thông thường người ta thường giải bài toán đối ngẫu (strong duality) . Bài toán đối ngẫu cũng là một QP nhưng nghiệm là sparse nên có những phương pháp giải hiệu quả hơn.

Một số sự tương tự giữa SVM và NN
![alt text](https://cdn.mathpix.com/snip/images/XFeNXnSwVrber9Q6PWSnjm7nb8WLC8TxkavH6nCKpQs.original.fullsize.png)





Source
[1](https://machinelearningcoban.com/2017/04/09/smv/)

**[Soft Margin SVM](https://machinelearningcoban.com/2017/04/13/softmarginsmv/#-tom-tat-va-thao-luan)**
* SVM thuần (Hard Margin SVM) hoạt động không hiệu quả khi có nhiễu ở gần biên hoặc thậm chí khi dữ liệu giữa hai lớp gần linearly separable. Soft Margin SVM có thể giúp khắc phục điểm này.
* Trong Soft Margin SVM, chúng ta chấp nhận lỗi xảy ra ở một vài điểm dữ liệu. Lỗi này được xác định bằng khoảng cách từ điểm đó tới đường biên tương ứng. Bài toán tối ưu sẽ tối thiểu lỗi này bằng cách sử dụng thêm các biến được gọi là slack varaibles.

**Kernel SVM**
* Nếu dữ liệu của hai lớp là không phân biệt tuyến tính (nonlinearly separable), chúng ta có thể tìm cách biến đổi dữ liệu sang một không gian mới sao cho trong không gian mới ấy, dữ liệu của hai lớp là phân biệt tuyến tính hoặc gần phân biệt tuyến tính.
* Việc tính toán trực tiếp hàm $\Phi()$ đôi khi phức tạp và tốn nhiều bộ nhớ. Thay vào đó, ta có thể sử dụng kernel trick. Trong cách tiếp cận này, ta chỉ cần tính tích vô hướng của hai vector bất kỳ trong không gian mới:$k(\mathbf{x}, \mathbf{z})=\Phi(\mathbf{x})^{T} \Phi(\mathbf{z})$

**Multi-class SVM**
* Giống như Softmax Regression, Multi-class SVM vẫn được coi là một bộ phân lớp tuyến tính vì đường phân chia giữa các class là các đường tuyến tính.
* Kernel SVM cũng hoạt động khá tốt, nhưng việc tính toán ma trận kernel có thể tốn nhiều thời gian và bộ nhớ. Hơn nữa, việc mở rộng nó ra cho bài toán multi-class classification thường không hiệu quả bằng Multi-class SVM. 
* Một ưu điểm nữa của Multi-class SVM là nó có thể được tối ưu bằng (Stochastic) Gradient Descnet, tức là nó phù hợp với các bài toán large-scale. Việc boundary giữa các class là tuyến tính có thể được giải quyết bằng cách kết hợp nó với Deep Neurel Networks. 




# 2.Instance-based (memory-based)

## K-nearest neighbors (KNN)
Không học mà chỉ nhớ một cách máy móc toàn bộ dữ liệu. KKN tìm đầu ra bằng thông tin của K điểm gần nhất.

Input dữ liệu đầu vào và số lượng  nhóm (K), với 1 điểm tính toán khoảng cách từ 1 điểm dữ liệu nhỏ nhất tới tất cả của dữ liệu đầu vào N điểm, và chọn ra K khoảng cách nhỏ nhất.

**2 vấn đề chính:**
- vấn đề về cách đo các điểm, thì các điểm thường được thể hiện bằng 1 vector đặc trưng, nên $l2-norm$ hay Euclid là cách đo khoảng cách thường được sử dụng
- vấn đề về tính toán, khi tập huấn luyện lớn và vector đặc trựng lớn $X \in R^{d*N}$. KNN sẽ phải tính toán khoảng cách từ 1 điểm dữ liệu nhỏ nhất tới tất cả N điểm, và chọn ra K khoảng cách nhỏ nhất (khối lượng tính toán lớn)

Xem thêm trang 103-104/ML cơ bản, để biết các optimize cách tính: *1. Khoảng cách từ 1 điểm tới từng điểm trong 1 tập hợp, 2. khoảng cách giữa từng cặp điểm trong 2 tập hợp*. Đối với GPU, xem thêm [faiss](https://github.com/facebookresearch/faiss)

Source: 
[1](https://machinelearningcoban.com/2017/01/08/knn/),

** Discuss & Code**
Ưu điểm của KNN:
 - Độ phức tạp tính toán của quá trình training là bằng 0.
 - Việc dự đoán kết quả của dữ liệu mới rất đơn giản.
 - Không cần giả sử gì về phân phối của các class.

Nhược điểm của KNN
 - KNN rất nhạy cảm với nhiễu khi K nhỏ.
 - Như đã nói, KNN là một thuật toán mà mọi tính toán đều nằm ở khâu test. Trong đó việc tính khoảng cách tới từng điểm dữ liệu trong training set sẽ tốn rất nhiều thời gian, đặc biệt là với các cơ sở dữ liệu có số chiều lớn và có nhiều điểm dữ liệu. Với K càng lớn thì độ phức tạp cũng sẽ tăng lên. Ngoài ra, việc lưu toàn bộ dữ liệu trong bộ nhớ cũng ảnh hưởng tới hiệu năng của KNN.

Tăng tốc cho KNN

Ngoài việc tính toán khoảng cách từ một điểm test data đến tất cả các điểm trong traing set (Brute Force), có một số thuật toán khác giúp tăng tốc việc tìm kiếm này. Bạn đọc có thẻ tìm kiếm thêm với hai từ khóa: K-D Tree và Ball Tree. Tôi xin dành phần này cho độc giả tự tìm hiểu, và sẽ quay lại nếu có dịp. Chúng ta vẫn còn những thuật toán quan trọng hơn khác cần nhiều sự quan tâm hơn.

**Code**

Sử dụng thư viện `sklearn.neighbors.KNeighborsClassifier()`

Mặc định `weights = 'uniform'`, nếu k = 10 thì vãi trò của mỗi điểm là như nhau để xét cho điểm hiện tại. 
- Để cải thiện chúng ta có thể đánh giá theo khoảng cách `weights = 'distance'` (càng lân cận càng tin tưởng)

In [0]:
# KNN    
from IPython.display import IFrame
IFrame(src='https://nbviewer.jupyter.org/github/tiepvupsu/ebookML_src/blob/master/src/knn/KNN.ipynb',
       width=800, height=600)

# 3.Clustering Algorithms


*   K-means clustering
*   List item



## K-means clustering
- unsupervised learning

Input là dữ liệu đầu vào và số lượng nhóm ($K$), tìm nhóm cho từng điểm dữ liệu và có $K$ nhóm

Source:
[1](https://machinelearningcoban.com/2017/01/01/kmeans/),
[2](https://machinelearningcoban.com/2017/01/04/kmeans2/)

![alt text](https://cdn.mathpix.com/snip/images/PT2D3fLs4a_7od0kWH8kWo1EPA41yKK6GI-HV0OBQ_4.original.fullsize.png)

**Tối ưu hàm mất mát**:
- Cố định M, tìm Y: nghĩa là giả sử các centroid, hay tìm các label vector để hàm mất mát nhỏ nhất.
- Cố định Y, tìm M: giả sử đã tìm được cluster cho từng điểm, hãy tìm centroid mới cho mỗi cluster đề hàm mất mát đạt giá trị nhỏ nhất


>**Thuật toán k-means clustering**:
- Input:Ma trận dữ liệu $X \in R^{d*N}$
- Output: ma trận centroid $M \in R^{d*K}$ và ma trận label $Y \in R^{N*K}$
>
>> 1.   Chọn K điểm bất kì làm trong training set là centroid ban đầu
2.   Phân mỗi điểm dữ liệu vào cluster có centroid gần nó nhất
3. Nếu việc phân nhóm dữ liệu vào từng cluster ở bước 2 không thay đổi so với vong lặp trước thì -> dừng thuật toán
4. Cập nhật centroid cho từng cluster: bằng cách lấy trung bình cộng của tất cả các điểm dữ liệu đã được gán vào cluster đó sau bước 2
5. Back -> step 2.



**Discuss & code**
Hạn chế
*    Chúng ta cần biết số lượng cluster cần clustering
*   Nghiệm cuối cùng phụ thuộc vào các centers được khởi tạo ban đầu
*    Các cluster cần có só lượng điểm gần bằng nhau
*    Các cluster cần có dạng hình tròn
*    Khi một cluster nằm phía trong 1 cluster khác



# 4.Reduce Dimension



### Singular Value Decomposition (SVD)
![alt text](https://cdn.mathpix.com/snip/images/JXD8GZE0T-iEvom84u0TFQl4QXzjm6cM2fC0n92kgFk.original.fullsize.png)

Chúng ta có thể đơn giản hóa giữ liệu bằng cách lựa chọn giá trị $k$ rank
Best rank là có thể được tính bằng xấp xỉ  Frobenius norm và norm 2.

Frobenius norm và norm 2 là hai norms được sử dụng nhiều nhất trong ma trận. Như vậy, xét trên cả hai norm này, Truncated SVD đều cho xấp xỉ tốt nhất. Vì vậy Truncated SVD còn được gọi là Best low-rank Approximation.



## Principal Component Analysis

Cách đơn giản nhất để giảm chiều dữ liệu từ $D$ về$K<D$ là chỉ giữ lại $K$ phần tử quan trọng nhất. Tuy nhiên, việc làm này chắc chắn chưa phải tốt nhất vì chúng ta chưa biết xác định thành phần nào là quan trọng hơn. Hoặc trong trường hợp xấu nhất, lượng thông tin mà mỗi thành phần mang là như nhau, bỏ đi thành phần nào cũng dẫn đến việc mất một lượng thông tin lớn.

Tuy nhiên, nếu chúng ta có thể biểu diễn các vector dữ liệu ban đầu trong một hệ cơ sở mới mà trong hệ cơ sở mới đó, tầm quan trọng giữa các thành phần là khác nhau rõ rệt, thì chúng ta có thể bỏ qua những thành phần ít quan trọng nhất.

---

**Thuật toán PCA** 

> ![alt text](https://cdn.mathpix.com/snip/images/aNj8bx_bo8CPSnN5AN3jPHiThzuei2ioOB27eJr6Aqk.original.fullsize.png)